# Example usage notebook of the DD-GAN repo

In [1]:
import numpy as np
import sklearn.preprocessing
import tensorflow as tf

After installation of the ddgan package run the following line to import it

In [2]:
from ddgan import GAN

In [6]:
kwargs = {
    "nsteps" : 5,
    "ndims" : 5,
    "lambda" : 10,
    "n_critic" : 5,
    "batches" : 10,
    "batch_size" : 20
}

gan = GAN()
gan.setup(kwargs)

nTrain = 100
nPOD = 10

ndims_latent_input = 2

t_begin = 0
t_end = nTrain - gan.nsteps + 1
training_data = np.zeros((t_end, nPOD * gan.nsteps), dtype=np.float32) # nTrain by nsteps*nPOD # 'float32' or np.float32

input_to_GAN = tf.random.normal([training_data.shape[0], ndims_latent_input])
input_to_GAN = input_to_GAN.numpy()

looking for previous saved models
making new generator and critic
